<a href="https://colab.research.google.com/github/hannahconnell/randomuser-data-extraction/blob/main/Coding_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import json
import duckdb
RU_URL = 'https://randomuser.me/api/'
GENDERIZE_URL = 'https://api.genderize.io'
API_KEY= '36b8d57defd066963ad04d9625c76613'

# Gather 250 user objects from RandomUser
response = requests.get(RU_URL, params ={'results': 250})
data = response.json()
df = pd.json_normalize(data['results'])

# Send list of names from RandomUser to Genderize, while taking advantage of Genderize's batch processing
gender_predictions = []
name_list = df['name.first'].tolist()
BATCH_SIZE = 10
for i in range(0, len(name_list), BATCH_SIZE):
  batch = name_list[i:i+BATCH_SIZE]
  params = [("name[]", name) for name in batch]
  params.append(("apikey", API_KEY))

  gender_response = requests.get(GENDERIZE_URL, params=params)
  gender_predictions.extend(gender_response.json())

gender_df = pd.DataFrame(gender_predictions)

# Merge gender_df to existing df and remove redundant name column
df = df.merge(gender_df, how="left", left_on="name.first", right_on="name")
df.drop(columns=["name"])

,gender_x,email,phone,cell,nat,name.title,name.first,name.last,location.street.number,location.street.name,...,registered.date,registered.age,id.name,id.value,picture.large,picture.medium,picture.thumbnail,count,gender_y,probability
0,male,jairo.rodriguez@example.com,(653) 143 4507,(613) 014 4162,MX,Mr,Jairo,Rodríguez,8786,Diagonal Herrera,...,2021-02-20T07:39:13.592Z,4,NSS,73 26 87 7646 4,https://randomuser.me/api/portraits/men/31.jpg,https://randomuser.me/api/portraits/med/men/31...,https://randomuser.me/api/portraits/thumb/men/...,128607,male,1.00
1,female,alexandra.mendoza@example.com,011-182-0198,081-984-0756,IE,Mrs,Alexandra,Mendoza,9217,O'Connell Street,...,2018-06-25T08:42:25.862Z,7,PPS,1853411T,https://randomuser.me/api/portraits/women/10.jpg,https://randomuser.me/api/portraits/med/women/...,https://randomuser.me/api/portraits/thumb/wome...,555453,female,0.99
2,male,roche.kempen@example.com,(063) 9785953,(06) 55934070,NL,Mr,Roché,Kempen,4384,Drentse Mondenweg,...,2016-02-11T19:17:08.157Z,9,BSN,65229397,https://randomuser.me/api/portraits/men/17.jpg,https://randomuser.me/api/portraits/med/men/17...,https://randomuser.me/api/portraits/thumb/men/...,32,male,0.56
3,male,arttu.ylitalo@example.com,05-107-458,041-787-96-52,FI,Mr,Arttu,Ylitalo,8831,Tahmelantie,...,2010-06-08T04:12:29.865Z,15,HETU,NaNNA079undefined,https://randomuser.me/api/portraits/men/88.jpg,https://randomuser.me/api/portraits/med/men/88...,https://randomuser.me/api/portraits/thumb/men/...,2876,male,1.00
4,male,felipe.reyes@example.com,934-799-601,634-916-874,ES,Mr,Felipe,Reyes,7821,Paseo de Zorrilla,...,2014-08-14T11:43:38.038Z,10,DNI,88711305-G,https://randomuser.me/api/portraits/men/72.jpg,https://randomuser.me/api/portraits/med/men/72...,https://randomuser.me/api/portraits/thumb/men/...,392178,male,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,female,mia.robinson@example.com,(111)-432-1384,(811)-677-7047,NZ,Ms,Mia,Robinson,6800,Blenheim Road,...,2019-05-31T12:34:46.860Z,6,,None,https://randomuser.me/api/portraits/women/54.jpg,https://randomuser.me/api/portraits/med/women/...,https://randomuser.me/api/portraits/thumb/wome...,141824,female,0.97
268,male,hubert.dupuis@example.com,075 718 71 42,077 995 00 82,CH,Monsieur,Hubert,Dupuis,4131,Rue Abel,...,2003-03-10T05:34:39.841Z,22,AVS,756.4061.4794.55,https://randomuser.me/api/portraits/men/12.jpg,https://randomuser.me/api/portraits/med/men/12...,https://randomuser.me/api/portraits/thumb/men/...,66963,male,0.99
269,female,violeta.ninkovic@example.com,028-4719-646,060-8293-183,RS,Miss,Violeta,Ninković,1648,Stevana Sremca,...,2019-02-09T22:34:47.060Z,6,SID,392334087,https://randomuser.me/api/portraits/women/76.jpg,https://randomuser.me/api/portraits/med/women/...,https://randomuser.me/api/portraits/thumb/wome...,62652,female,0.99
270,male,aaly.aalyzdh@example.com,024-05216042,0995-960-0749,IR,Mr,علی,علیزاده,1824,موحد دانش,...,2018-10-30T00:49:46.782Z,6,,None,https://randomuser.me/api/portraits/men/63.jpg,https://randomuser.me/api/portraits/med/men/63...,https://randomuser.me/api/portraits/thumb/men/...,13760,male,0.97


In [ ]:
# First query: What percentage of genders are correctly predicted?
duckdb.sql(
    "SELECT (SUM(CASE WHEN gender_x = gender_y THEN 1 ELSE 0 END) * 100.00 / COUNT(*)) AS pct_correct FROM df"
    )

┌───────────────────┐
│    pct_correct    │
│      double       │
├───────────────────┤
│ 96.32352941176471 │
└───────────────────┘

In [ ]:
# Second query: On average, which gender is predicted more accurately?
duckdb.sql(
    "SELECT AVG(CASE WHEN (gender_x = gender_y AND gender_x = 'male') THEN 1 ELSE 0 END) AS Male, AVG(CASE WHEN (gender_x = gender_y AND gender_x = 'female') THEN 1 ELSE 0 END) AS Female FROM df"
    )

┌────────┬────────────────────┐
│  Male  │       Female       │
│ double │       double       │
├────────┼────────────────────┤
│    0.5 │ 0.4632352941176471 │
└────────┴────────────────────┘

In [ ]:
# Third query: What are the 5 names with the lowest probability score?
duckdb.sql(
    "SELECT name FROM df ORDER BY probability ASC LIMIT 5"
    )

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ Nehrika    │
│ Tonkostana │
│ Nandielly  │
│ Michon     │
│ Roché      │
└────────────┘

In [ ]:
# Fourth query: How many instances are there where the probability is lower than average, and the number of rows searched is higher than average?
duckdb.sql(
    "WITH avgs AS (SELECT AVG(probability) AS avg_probability, AVG(count) AS avg_rows FROM df) SELECT name, probability, count FROM df, avgs WHERE df.probability < avgs.avg_probability AND df.count > avgs.avg_rows"
)

┌─────────┬─────────────┬─────────┐
│  name   │ probability │  count  │
│ varchar │   double    │  int64  │
├─────────┼─────────────┼─────────┤
│ Alex    │        0.95 │ 1665200 │
│ Sam     │         0.9 │  645671 │
│ Andrea  │        0.69 │ 1672295 │
└─────────┴─────────────┴─────────┘

In [ ]:
# Fifth query: Which gender is harder to predict?
duckdb.sql("WITH avgs AS (SELECT AVG(probability) AS avg_probability FROM df) SELECT SUM(CASE WHEN(gender_x = 'male') THEN 1 ELSE 0 END) AS Male, SUM(CASE WHEN(gender_x = 'female') THEN 1 ELSE 0 END) AS Female FROM df, avgs WHERE df.probability < avgs.avg_probability")

┌────────┬────────┐
│  Male  │ Female │
│ int128 │ int128 │
├────────┼────────┤
│     20 │     23 │
└────────┴────────┘

┌────────────────────┐
│  avg(probability)  │
│       double       │
├────────────────────┤
│ 0.9539705882352959 │
└────────────────────┘